<a href="https://colab.research.google.com/github/RaymondSalim/google-drive-scripts/blob/master/Google_Drive_Scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Execute this code first before anything else
### **DON'T FORGET TO PASTE YOUR FILE ID BELOW IF YOU ARE UPLOADING client_id.json FROM GOOGLE DRIVE**





In [ ]:
#@title
file_id = "" #@param {type:"string"}
!gdown --id file_id
from __future__ import print_function


import io
import sys
import pathlib
import requests
import shutil
import os
import time
from os import system
from pathlib import Path
from googleapiclient import discovery
from httplib2 import Http
from oauth2client import file, client, tools
from googleapiclient.http import MediaIoBaseDownload
from oauth2client.tools import argparser, run_flow

class googlescripts:
  def download(self, fId, fName, teamDrive, finalDirectory, teamDriveId):
    # Start
      global fileCount, totalSize 
      if (teamDrive):
        fileList = DRIVE.files().list(
              driveId = teamDriveId,
              corpora = "teamDrive",
              supportsAllDrives = 'true',
              includeItemsFromAllDrives = 'true',
              q="('" + fId + "' in parents)"
            ).execute().get('files', [])
      else:
        fileList = DRIVE.files().list(
              corpora = "user",
              q="('" + fId + "' in parents)"
            ).execute().get('files', [])

      print("\n\n")
      for f in fileList:
        finalDir = finalDirectory
        print(f['name'] + " " + f['id'])

        if f['mimeType'] == 'application/vnd.google-apps.folder':
            initDir = os.getcwd()
            dir = os.path.join( initDir , f['name'])
            
            if not os.path.exists(dir):
              os.mkdir(dir)

            os.chdir(dir)
            print("\n\nFiles in " + f['name'])
            self.download(f['id'], f['name'], teamDrive, finalDirectory, teamDriveId)

            os.chdir(initDir)
        
        else:

          a = {}
          a[f['name']] = f['id']

          file_id = f['id']
          local_filename = f['name']

          fileDir = str(pathlib.Path().absolute().resolve()) + "/" + local_filename
          shortFileDir = fileDir[8::]
          finalDir = finalDir + shortFileDir

          if (os.path.isfile(os.path.normpath(finalDir))):
            print("File exists in destination, Skipping \n")
            continue
            
          try:
            request = DRIVE.files().get_media(fileId=file_id)
            fh = io.FileIO(local_filename, 'wb')
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                print( "Download %d%%." % int(status.progress() * 100))
            print(local_filename + " downloaded " + str(pathlib.Path().absolute().resolve()))
            fileCount += 1
            totalSize += os.path.getsize(fileDir)


            
            ### FILE COPYING ###
            # fileDir = str(pathlib.Path().absolute().resolve()) + "/" + local_filename
            # shortFileDir = fileDir[8::]
            # finalDir = finalDir + shortFileDir
            print("input = " + fileDir)
            print("output = " + finalDir)
            temp = os.path.normpath(finalDir.replace(local_filename , ""))

            if not os.path.exists(temp):
              os.makedirs(temp)

            print("Start moving file")
            shutil.move(os.path.normpath(fileDir), temp)
            print("File moved\n")
          
          except OSError as error:
            print("Failed to get file size)")
          
          except Exception as error:
            print(error)
            print("Error file: " + local_filename + "   " + file_id)
            print("Skipping file\n")
            driveErrors[local_filename] = file_id
            continue

    #### END ####

  def downloadSearch(self, fId, fName, teamDrive, finalDirectory, name, teamDriveId):
    # Start
      global fileCount, totalSize 
      if (teamDrive):
        fileList = DRIVE.files().list(
              driveId = teamDriveId,
              corpora = "teamDrive",
              supportsAllDrives = 'true',
              includeItemsFromAllDrives = 'true',
              q="(name contains '" + name + "')"
            ).execute().get('files', [])
      else:
        fileList = DRIVE.files().list(
              corpora = "user",
              q="(name contains '" + name + "')"
            ).execute().get('files', [])

      print("\n\n")
      for f in fileList:
        finalDir = finalDirectory
        print(f['name'] + " " + f['id'])

        if f['mimeType'] == 'application/vnd.google-apps.folder':
            initDir = os.getcwd()
            dir = os.path.join( initDir , f['name'])
            
            if not os.path.exists(dir):
              os.mkdir(dir)

            os.chdir(dir)
            print("\n\nFiles in " + f['name'])
            self.download(f['id'], f['name'], teamDrive, finalDirectory, teamDriveId)

            os.chdir(initDir)
        
        else:

          a = {}
          a[f['name']] = f['id']

          file_id = f['id']
          local_filename = f['name']

          fileDir = str(pathlib.Path().absolute().resolve()) + "/" + local_filename
          shortFileDir = fileDir[8::]
          finalDir = finalDir + shortFileDir

          if (os.path.isfile(os.path.normpath(finalDir))):
            print("File exists in destination, Skipping \n")
            continue
          
          try:
            request = DRIVE.files().get_media(fileId=file_id)
            fh = io.FileIO(local_filename, 'wb')
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                print( "Download %d%%." % int(status.progress() * 100))
            print(local_filename + " downloaded " + str(pathlib.Path().absolute().resolve()))
            fileCount += 1
            totalSize += os.path.getsize(fileDir)


            
            ### FILE COPYING ###
            # fileDir = str(pathlib.Path().absolute().resolve()) + "/" + local_filename
            # shortFileDir = fileDir[8::]
            # finalDir = finalDir + shortFileDir
            print("input = " + fileDir)
            print("output = " + finalDir)
            temp = os.path.normpath(finalDir.replace(local_filename , ""))

            if not os.path.exists(temp):
              os.makedirs(temp)

            print("Start moving file")
            shutil.move(os.path.normpath(fileDir), temp)
            print("File moved\n")
          
          except OSError as error:
            print("Failed to get file size)")

          except Exception as error:
            print(error)
            print("Error file: " + local_filename + "   " + file_id)
            print("Skipping file\n")
            driveErrors[local_filename] = file_id
            continue


    #### END ####
          
  def renameTeamDrive(self, fName, removeName, folderId, wholeFolder):
      if wholeFolder:
        fileList = DRIVE.files().list(
              driveId = teamDriveId,
              corpora = "teamDrive",
              supportsAllDrives = 'true',
              includeItemsFromAllDrives = 'true',
              q="(name contains '" + fName + "')"
            ).execute().get('files', [])
      else:
        fileList = DRIVE.files().list(
              driveId = teamDriveId,
              corpora = "teamDrive",
              supportsAllDrives = 'true',
              includeItemsFromAllDrives = 'true',
              q="(name contains '" + fName + "' AND '"+ folderId +"' in parents)"
            ).execute().get('files', [])

      for f in fileList:
        print(f['name'] + "  " + f['id'])

      for f in fileList:
        originalName = f['name']
        originalNameLower = f['name'].lower()
        firstIndex = originalNameLower.index(fName.lower())
        newName = originalName[0:firstIndex:] + removeName + originalName[firstIndex + len(fName) : len(originalName):]
        DRIVE.files().update(
            fileId = f['id'],
            supportsAllDrives = 'true',
            body = {
              "name": newName,
              "driveId": teamDriveId
            }
        ).execute()
        print(newName)

  def renameDrive(self, fName, removeName, folderId, wholeFolder):
      if wholeFolder:
          fileList = DRIVE.files().list(
          corpora = "user",
          q="(name contains '" + fName + "')"
        ).execute().get('files', [])
      else:
        fileList = DRIVE.files().list(
          corpora = "user",
          q="(name contains '" + fName + "' AND '"+ folderId +"' in parents)"
        ).execute().get('files', [])

      for f in fileList:
        print(f['name'] + "  " + f['id'])

      for f in fileList:
        originalName = f['name']
        originalNameLower = f['name'].lower()
        firstIndex = originalNameLower.index(fName.lower())
        newName = originalName[0:firstIndex:] + removeName + originalName[firstIndex + len(fName) : len(originalName):]
        DRIVE.files().update(
            fileId = f['id'],
            body = {
                "name": newName
            }
        ).execute()
        print(newName)



###############################
sys.argv = ['--noauth_local_webserver']
args = argparser.parse_args()
args.noauth_local_webserver = True

os.chdir('/content')

initDirectory = os.getcwd()
googleclass = googlescripts()

SCOPES = 'https://www.googleapis.com/auth/drive'
store = file.Storage('storage.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id.json', SCOPES)
    creds = tools.run_flow(flow, store, args)
DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))

driveErrors = {}


system('clear') 


---

# Google Copy **FROM** TeamDrive Script



Download any files/folders as long as you have viewing access and copies it to your drive




In [ ]:
#@title

from google.colab import drive
drive.mount('/gdrive')

In [ ]:
#@title

os.chdir('/content')

drives = DRIVE.drives().list(pageSize=100).execute().get('drives')

for i in range(1, len(drives)):
  print(f"{i}. {drives[i]['name']}")

driveIDs = []

while True:
  driveInput = input("Enter the number of the drive you want to search from or type \"exit\" or type \"all\"\n")
  if driveInput == "exit":
    break
  elif driveInput == "all":
    driveIDs = drives
    break
  else:
    driveIDs.append(drives[int(driveInput)])
  
folderId = input("Please enter source folder ID, or root for the whole drive\n")
folderName = input("Please enter the folder name for the downloaded files \n")
nameQuery = input("Please enter search query or leave blank \n")
finalDest = input("Where do you want to move the files to? Please enter the directory. \n ex: \"/gdrive/Shared drives/MyTeamDrive\"\n")

startTime = time.time()
fileCount = 0
totalSize = 0

for drive in driveIDs:
  tempDir = os.path.join( os.getcwd(), folderName)
  if not (os.path.exists(tempDir)):
      os.mkdir(tempDir)
  os.chdir(tempDir)

  if (len(nameQuery) > 0):
    googleclass.downloadSearch(folderId, folderName, True, finalDest, nameQuery, drive['id'])
  else:
    googleclass.download(folderId, folderName, True, finalDest, drive['id'])

  os.chdir(initDirectory)


finalTime = time.time() - startTime

hours = int(finalTime / 3600)
finalTime %= 3600

minutes = int(finalTime / 60)
finalTime %= 60

seconds = int(finalTime)

giga = int(totalSize / 1073741824)
totalSize %= 1073741824

mega = int(totalSize / 1048576)



print(f"{fileCount} files downloaded (Total size: {giga}.{mega}GB) \nFiles downloaded in {hours} hours {minutes} minutes {seconds} seconds")



print("File Cleanup")
shutil.rmtree(tempDir, True, )




errorTotal = len(driveErrors)
print(f"{errorTotal} errors")



---

# Google Copy **FROM** Non - TeamDrive Script



Download any files/folders as long as you have viewing access and copies it to your drive



In [ ]:
#@title

folderId = input("Please enter source folder ID\n")
folderName = input("Please enter the folder name for the downloaded files \n")
finalDest = input("Where do you want to move the files to? Please enter the directory. \n ex: \"/gdrive/Shared drives/MyTeamDrive\"\n")


tempDir = os.path.join( os.getcwd(), folderName)
if not (os.path.exists(tempDir)):
    os.mkdir(tempDir)
os.chdir(tempDir)


startTime = time.time()
fileCount = 0
totalSize = 0


googleclass.download(folderId, folderName, False, finalDest)


finalTime = time.time() - startTime

hours = finalTime / 3600
finalTime %= 3600

minutes = finalTime / 60
finalTime %= 60

seconds = finalTime

giga = totalSize / 1073741824
totalSize %= 1073741824

mega = totalSize / 1048576



print(f"{fileCount} files downloaded (Total size: {giga}.{mega}GB) \nFiles downloaded in {hours} hours {minutes} minutes {seconds} seconds")

print("File Cleanup")
shutil.rmtree(tempDir, True, )

os.chdir(initDirectory)

errorTotal = len(driveErrors)
print(f"{errorTotal} errors")

---
# Google Drive Batch Rename Script

You can rename multiple files at once such as removing 'Copy of'

In [ ]:
#@title

SCOPES = 'https://www.googleapis.com/auth/drive'

isTeamDrive = input("Are the files located in a Team Drive/Shared Drives? (Yes / No)")

while not (isTeamDrive[0].lower() == "y" or isTeamDrive[0].lower() == "n"):
  isTeamDrive = input("Are the files located in a Team Drive/Shared Drives? (Yes / No)")

if (isTeamDrive[0].lower() == "y"):
  teamDriveId = input("Please enter TeamDrive ID\n")
  removeString = input("Please input the string you would like to replace")
  newString = input("What do you want to replace it with?")
  folder = input("What is the folder ID? (Ignore if you would like to search the whole drive)\n")
  wholeFolder = False
  if len(folder) == 0:
    wholeFolder = True
  googleclass.renameTeamDrive(removeString, newString, folder, wholeFolder)

elif (isTeamDrive[0].lower() == "n"):
  removeString = input("Please input the string you would like to replace\n")
  newString = input("What do you want to replace it with?\n")
  folder = input("What is the folder ID? (Ignore if you would like to search the whole drive)\n")
  wholeFolder = False
  if len(folder) == 0:
    wholeFolder = True
  googleclass.renameDrive(removeString, newString, folder, wholeFolder)







